<a href="https://colab.research.google.com/github/AvisR/Drawdown/blob/main/Drawdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question - Find N largest drawdowns given the return series and N

Defining Drawdown - Largest negative movement in percentage. 2nd largest drawdown will be the 2nd largest negative movement in percentage. 

Drawdown = (Peak - Trough)/Peak

Return series = rets

N = N

---



In [92]:
#INPUTS - Create a list (10) of random returns from (-5% and 10%)
#This will not be needed if we have a list of our own returns.
#II = Initial Investment

II = 1
N = 1

import random
rets = random.sample(range(-50,100),100)
rets = [x/1000 for x in rets]

print(rets)

[-0.017, -0.011, 0.019, -0.037, 0.018, 0.075, 0.066, 0.072, 0.058, 0.057, -0.024, 0.088, 0.09, -0.006, -0.015, 0.025, -0.036, 0.026, 0.084, 0.027, 0.098, 0.089, -0.048, 0.003, 0.023, 0.045, 0.038, -0.039, 0.01, 0.014, 0.053, -0.044, 0.02, -0.009, -0.027, 0.082, 0.05, 0.056, 0.011, 0.069, 0.042, 0.032, 0.022, 0.04, 0.071, 0.008, 0.078, -0.018, -0.042, 0.052, 0.08, 0.068, 0.063, 0.037, 0.062, -0.032, 0.096, -0.016, 0.016, 0.081, -0.041, -0.023, 0.067, -0.007, -0.047, 0.048, -0.021, 0.041, -0.003, 0.094, -0.03, 0.064, -0.05, -0.045, 0.093, 0.039, 0.059, 0.004, -0.014, -0.049, 0.061, 0.047, -0.019, 0.024, 0.051, 0.001, -0.025, 0.002, 0.007, -0.005, -0.008, -0.035, 0.044, 0.036, 0.077, 0.0, -0.038, -0.043, -0.028, 0.083]


In [93]:
#Step 1 = Calculate value of portfolio.
#Initial investment = 1, value of the portfolio is calculated using initial investment and cumulative product.
#For eg. Value of the portfolio at the end of period 10 would be (1*(1+p1 return)*(1+p2 return)*....(1+p10 return))

cmfactor = [0]*(len(rets))

def portvalue(rets):
    mfactor = [1+x for x in rets]
    for i in range(0,len(rets)):
      if i == 0:
        cmfactor[i] = II * mfactor[i]
      else:
        cmfactor[i] = cmfactor[i-1]*mfactor[i]
      cmfactor[i] = round(cmfactor[i],4)
    return(cmfactor)

portvalue = portvalue(rets)

print(portvalue)


[0.983, 0.9722, 0.9907, 0.954, 0.9712, 1.044, 1.1129, 1.193, 1.2622, 1.3341, 1.3021, 1.4167, 1.5442, 1.5349, 1.5119, 1.5497, 1.4939, 1.5327, 1.6614, 1.7063, 1.8735, 2.0402, 1.9423, 1.9481, 1.9929, 2.0826, 2.1617, 2.0774, 2.0982, 2.1276, 2.2404, 2.1418, 2.1846, 2.1649, 2.1064, 2.2791, 2.3931, 2.5271, 2.5549, 2.7312, 2.8459, 2.937, 3.0016, 3.1217, 3.3433, 3.37, 3.6329, 3.5675, 3.4177, 3.5954, 3.883, 4.147, 4.4083, 4.5714, 4.8548, 4.6994, 5.1505, 5.0681, 5.1492, 5.5663, 5.3381, 5.2153, 5.5647, 5.5257, 5.266, 5.5188, 5.4029, 5.6244, 5.6075, 6.1346, 5.9506, 6.3314, 6.0148, 5.7441, 6.2783, 6.5232, 6.9081, 6.9357, 6.8386, 6.5035, 6.9002, 7.2245, 7.0872, 7.2573, 7.6274, 7.635, 7.4441, 7.459, 7.5112, 7.4736, 7.4138, 7.1543, 7.4691, 7.738, 8.3338, 8.3338, 8.0171, 7.6724, 7.4576, 8.0766]


In [95]:
# Step 2 = Find all difference. 
# The idea is to create a matrix which contains differences betweem all portfolio value. 
# For eg - The first row of the matrix will contain difference between initial investment and the subsequent values. 
# This can be optimized futher if we stop the iteration as soon we find a value greater than the value we are comparing as any drawdown found after that will not be the max drawdown.
# For eg. Value at t0 = 1. Value at time 10 = 1.3 Value at time 23 = .5. The drawdown from time 10 will always be greater than the drawdown from t0. So we need to evaluate only till t9 for t0.

#Matrix of differences

ddmatrix = [[0 for i in range(len(rets))] for a in range(len(rets))]

def drawdownmat(rets):
    for i in range(len(rets)):
      if i == 0:
        for a in range(len(rets)):
         ddmatrix[i][a]= (portvalue[a]-II)/II
      else:
        for a in range(len(rets)):
         ddmatrix[i][a]= (portvalue[a]-portvalue[i-1])/portvalue[i-1]
    return(ddmatrix)

ddmatrix = drawdownmat(rets)

for i in range(len(rets)):
  for a in range(len(rets)):
    if a < i:
      ddmatrix[i][a] = 0
    else:
      ddmatrix[i][a] = round(ddmatrix[i][a],4)


In [96]:
#Step 3 = Find the largest drawdown for each date. 

mddmatrix = [[0 for i in range(3)] for a in range(len(rets))]

def min_value(ddmatrix):
      l = len(ddmatrix)
      minlist = [min((ddmatrix)[i]) for i in range(l)]
      mdd = minlist
      return (minlist)

mdd = min_value(ddmatrix)

In [97]:
#Step 4 - Build MDD Matrix(Maximum drawdown for each date along with the time period). 
#Remove 0s and overlapping drawdowns and return Nth drawdown. 

mddmatrix = [[0 for i in range(3)] for a in range(len(rets))]

#Build MDD matrix and remove Zero drawdowns
for x in range(len(mddmatrix)):
  mddmatrix[x][0] = mdd[x]
  for a in range(len(rets)):
    if ddmatrix[x][a] == mddmatrix[x][0]:
       mddmatrix[x][1] = x
       mddmatrix[x][2] = a+1
    if mddmatrix[x][0] >= 0: 
       mddmatrix[x][1] = 0
       mddmatrix[x][2] = 0

mddmatrix = ([i for i in mddmatrix if any(i)])
mddmatrix.sort()

#Remove overlapping drawdowns
def unique_items(L): 
    found = set()
    for item in L:
        if item[2] not in found:
            yield item
            found.add(item[2])

mddmatrix = list(unique_items(mddmatrix))
#nth drawdown
def ndraw(N):
  if N > len(mddmatrix):
    return(print("We do not have", N, "drawdown"))
  elif mddmatrix[1] == 0:
    return(print("We do not have", N, "drawdown"))
  else:
    return(mddmatrix[N-1])

ndrawdown = ndraw(N)

print(N, "th largest drawdown is", ndrawdown[0], "and it occurs between", ndrawdown[1], "and", ndrawdown[2])

1 th largest drawdown is -0.1051 and it occurs between 95 and 99
